In [1]:
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions

import itertools

import numpy as np
import matplotlib.pyplot as plt
tf.enable_eager_execution()

In [ ]:
def build_net():
    return tf.keras.Sequential([
        tf.keras.layers.LSTM(64, activation=tf.nn.selu, return_sequences=True),
#         tf.keras.layers.Dense(64, activation=tf.nn.selu),
#         tf.keras.layers.Dense(64, activation=tf.nn.selu),
        tf.keras.layers.Dense(1),
        tf.keras.layers.Flatten()
    ])

In [ ]:
def discount_rewards(rewards, gamma):
    rewards = reversed(tf.unstack(rewards))
    return tf.stack(list(reversed(list(itertools.accumulate(rewards, lambda x, y: x*gamma+y)))), axis=0)
#     return tf.expand_dims(tf.accumulate_n(lambda x, y: x+gamma*y, tf.transpose(rewards)), -1)

class Critics():
    """
    Train an ensemble of critics at different time scales.
    Same input (extension - could be different inputs...)
    Different targets
    """
    # oh shit! this is going to turn into something like GVFs!?!?!? sick!
    def __init__(self, n_critics):
        self.fns = [build_net() for _ in range(n_critics)]
    
    def __call__(self, state):
        # want to calculate this more efficiently.
        # maybe using heirarchical LSTMs? or GVFs/cumulants!?
        return tf.add_n([fn(state) for fn in self.fns])
    
    def get_loss(self, state, r):
        vs = [fn(state) for fn in self.fns]
        rs = [discount_rewards(r, g) for g in [0.999, 0.99, 0.9]]
        return tf.add_n([tf.losses.mean_squared_error(r, v) for r, v in zip(vs, rs)])

In [ ]:
critics = Critics(3)

states = tf.random_normal([5, 20, 3])
rs = tf.random_normal([5, 20])
v = critics(states)
loss = critics.get_loss(states, rs)

In [ ]:
loss

In [ ]:
# So I should be able to use this in a vanilla A2C framework?
# Does it help!?

# Ok, but want HRL! How can we choose actions in a heirarchical manner?

In [ ]:
def discount_rewards(rewards, gamma):
    rewards = tf.unstack(rewards)
    return tf.stack(list(reversed(list(itertools.accumulate(list(reversed(rewards)), lambda x, y: x+gamma*y)))), axis=0)

In [ ]:
def discount_rewards(rewards, gamma):
    rewards = tf.unstack(rewards)
    return tf.stack(list(reversed(list(itertools.accumulate(list(reversed(rewards)), lambda x, y: x+gamma*y)))), axis=0)

In [ ]:
y = tf.reshape(y, [1, 1000, 1])
w = tf.exp(tf.reshape(tf.constant(range(10), dtype=tf.float32), [10, 1, 1]))
z = tf.nn.conv1d(y, w, 1, "SAME")

In [ ]:
x = tf.linspace(0.0,12.0, 1000)
y = tf.sin(x*3)+tf.sin(x*4+0.2) + tf.sin(x*5+0.5)

In [ ]:
plt.plot(x, y)

In [ ]:
plt.plot(x, z[0, :, 0])

In [4]:
def concrete(x):
    n = tf.random_uniform(minval=0, maxval=1, dtype=tf.float32, shape=tf.shape(x)[0])
    g = -tf.log(-tf.log(n))
    return tf.argmax(tf.log(x) + g, axis=-1)

class ActorCritic():
    def __init__(self, n_actions):
        self.nn = tf.keras.Sequential([
            tf.keras.layers.Dense(64, activation=tf.nn.selu),
            tf.keras.layers.Dense(n_actions+1),
        ])
        
        
    def __call__(self, x):
        z = self.nn(x)
        return z[..., :-1], z[..., -1:]
    
    def loss(self, x, a, r):
        pass

In [10]:
learners = [ActorCritic(4) for _ in range(4)]

In [18]:
s = tf.random_normal([1, 8])
logits, vs = [tf.add_n(arr) for arr in zip(*[l(s) for l in learners])]

In [19]:
logits

<tf.Tensor: id=717, shape=(1, 4), dtype=float32, numpy=
array([[ 0.46686375, -0.72043324,  2.715553  ,  1.7438973 ]],
      dtype=float32)>

In [ ]:
# what am I trying to build here.
# what bias am I trying to encode?
# use low freq when possible?